In [ ]:
import cv2
import mediapipe as mp
import math

cap = cv2.VideoCapture(1)  # Поменял на камеру 0, так как обычно она представлена первой
c = 0
state = False
# Инициализация модели holistic
with mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Перекрасить изображение
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Детектирование
        results = holistic.process(image)

        # Перекрасить изображение обратно в BGR для отрисовки
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            # Выделение ключевых точек
            if results.pose_landmarks.landmark:
                # Точка 25
                left = results.pose_landmarks.landmark[23]
                left_x = int(left.x * image.shape[1])
                left_y = int(left.y * image.shape[0])
                cv2.circle(image, (left_x, left_y), 5, (255, 255, 255), -1)  # Белый цвет

                # Точка 26
                right = results.pose_landmarks.landmark[27]
                right_x = int(right.x * image.shape[1])
                right_y = int(right.y * image.shape[0])
                cv2.circle(image, (right_x, right_y), 5, (255, 255, 255), -1)  # Белый цвет

                # Вычисление вектора от точки 25 к точке 26
                vector_x = right_x - left_x
                vector_y = right_y - left_y

                # Вычисление угла в градусах
                angle_degrees = math.degrees(math.atan2(vector_y, vector_x))

                # Отображение угла на изображении
                cv2.putText(image, f'Angle: {angle_degrees:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                if angle_degrees > 62 and angle_degrees < 70 and state or angle_degrees > 110 and angle_degrees < 120 and state:
                    c += 1
                    cv2.putText(image, f'Count: {c}', (70, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    cv2.putText(image, 'Good', (40, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    state = False
                elif not state:
                    # c = 1
                    print(c)
                    cv2.putText(image, f'Count: {c}', (70, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    cv2.putText(image, 'Bad', (40, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    state = True


            cv2.imshow('Raw Webcam Feed', image)
        except:
            print('')
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
